# LAB-18-01 產生交易測試資料(Claude AI)
由 Claude AI 生成  

[提示詞](https://claude.ai/share/56c5df98-8018-4069-9488-19cdb2f6e600)：  
```
請提供 Python 範例程式產生測試資料，並儲存為 CSV 檔，測試資料欄位有["交易時間","交易主機","交易代碼","交易延遲","回傳碼"]，各欄位說明如下：
1. "交易時間"：格式為 "YYYY-MM-DD hh:mm:ss.SSS" 
2. "交易主機"：值清單為 ["AP1","AP2"]
3. "交易代碼"：值清單為 ["A001","B002","C003","D004"]
4. "交易延遲"：數值格式為 int ，值介於 100 - 9999999
5. "回傳碼"：值清單為 ["0000","X999"]。
請產出 500000 測試資料，其中 
* "交易主機"為亂數分佈
* "交易代碼"之"A001"佔比45%，其他為亂數分佈
* "交易時間"為 "2025-03-12 06:00:00.000" 至 "2025-03-12 18:00:00.000" 
* "交易延遲" 的平均值約為 500 ，中間值約為1000     
* "回傳碼" 為 "0000" 佔比約為 99%
```


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import csv

In [ ]:
# 設定隨機種子以確保結果可重現
np.random.seed(42)
random.seed(42)

In [ ]:
# 定義常數
TOTAL_RECORDS = 500_000
fname = "test_data1.csv"

In [ ]:
# 定義各欄位的值域
HOSTS = ["AP1", "AP2"]
TRANSACTION_CODES = ["A001", "B002", "C003", "D004"]
RESPONSE_CODES = ["0000", "X999"]

# 產生交易時間
start_time = datetime.strptime("2025-03-12 06:00:00.000", "%Y-%m-%d %H:%M:%S.%f")
end_time = datetime.strptime("2025-03-12 18:00:00.000", "%Y-%m-%d %H:%M:%S.%f")
total_seconds = (end_time - start_time).total_seconds()

In [ ]:
# 產生隨機時間列表
random_seconds = np.sort(np.random.uniform(0, total_seconds, TOTAL_RECORDS))
transaction_times = [
    (start_time + timedelta(seconds=sec)).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    for sec in random_seconds
]

In [ ]:
# 產生交易主機 (隨機分佈)
hosts = np.random.choice(HOSTS, TOTAL_RECORDS)

In [ ]:
# 產生交易代碼 (A001佔45%，其他均分)
# 計算其他代碼的機率
other_code_prob = (1 - 0.45) / (len(TRANSACTION_CODES) - 1)
code_probabilities = [0.45 if code == "A001" else other_code_prob for code in TRANSACTION_CODES]
transaction_codes = np.random.choice(TRANSACTION_CODES, TOTAL_RECORDS, p=code_probabilities)

In [ ]:
# 產生交易延遲 (平均值約500，中位數約1000)
# 使用指數分佈來滿足這個要求
# 調整參數使平均值約為500，中位數約為1000
scale = 500  # 調整這個參數來控制分佈
delays = np.random.exponential(scale, TOTAL_RECORDS)
# 將值限制在100到9999999之間
delays = np.clip(delays, 100, 9999999)
# 轉為整數
delays = delays.astype(int)

In [ ]:
# 產生回傳碼 (0000佔99%)
response_codes = np.random.choice(RESPONSE_CODES, TOTAL_RECORDS, p=[0.99, 0.01])

In [ ]:
# 創建DataFrame
df = pd.DataFrame({
    "交易時間": transaction_times,
    "交易主機": hosts,
    "交易代碼": transaction_codes,
    "交易延遲": delays,
    "回傳碼": response_codes
})

# 計算統計資訊以驗證結果
print(f"總記錄數: {len(df)}")
print(f"交易主機分佈:\n{df['交易主機'].value_counts(normalize=True) * 100}")
print(f"交易代碼分佈:\n{df['交易代碼'].value_counts(normalize=True) * 100}")
print(f"交易延遲平均值: {df['交易延遲'].mean()}")
print(f"交易延遲中位數: {df['交易延遲'].median()}")
print(f"回傳碼分佈:\n{df['回傳碼'].value_counts(normalize=True) * 100}")

In [ ]:

# 將資料儲存為CSV檔
df.to_csv(fname, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"已成功將測試資料儲存至 {fname}")

# 額外：驗證交易時間的分佈
min_time = df["交易時間"].min()
max_time = df["交易時間"].max()
print(f"交易時間範圍: {min_time} 至 {max_time}")

In [ ]:
#檢視資料內容

# Windows 
#!dir {fname}
#!more -n 20 {fname} 

# Mac OSX
!ls -ltr {fname} 
!head {fname}

In [ ]:
# TODO
# 修改提示詞，將 "交易時間" 排序後再儲存